In [ ]:
import pandas as pd
import glob
from igc_lib.igc_lib import Flight
from pdflight import flight_to_dataframe,thermals_to_dataframe
from joblib import Parallel, delayed

flights = Parallel(n_jobs=-1)(delayed(Flight.create_from_file)(i) for i in glob.glob("igc_files/*.igc"))
#flight = Flight.create_from_file("/home/muenchm/thinclient_drives/C:/Users/muenchm/Desktop/2025-05-01 Wolfgang Janowitsch 562460.igc")


In [ ]:
fl = pd.concat(Parallel(n_jobs=-1)(delayed(flight_to_dataframe)(flight) for flight in flights)).sort_index()
fl

In [ ]:
th = pd.concat(thermals_to_dataframe(flight)[1:] for flight in flights).sort_index().loc[fl.loc[fl.index, "task"]]
th

In [ ]:
def shift_datetime(i, dt):
    return ((i[0] + dt),) + i[1:]

In [ ]:
df = pd.DataFrame()
df["t"] = th.index.get_level_values("datetime") + th
df = df.sort_index()
for i in th.index:
    df.loc[i, "v"] = (fl.loc[shift_datetime(i, th.loc[i])].alt - fl.loc[i].alt) / th.loc[i].total_seconds()
    df.loc[i, "n"] = (fl.loc[shift_datetime(i, pd.Timedelta(seconds=40))].alt - fl.loc[i].alt) / 40
    df.loc[i, "x"] = (fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=60))].alt - fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=20))].alt) / -40
df["dn"] = df.n / df.v
df["dx"] = df.x / df.v

In [ ]:
print(df[df.v > 1].groupby(["code", "id"]).dn.mean().sort_values())

In [ ]:
print(df[df.v > 1].groupby(["code", "id"]).dx.mean().sort_values())

In [ ]:
ll = pd.concat([df[df.v > 1].groupby(["code", "id"]).dn.mean(),df[df.v > 1].groupby(["code", "id"]).dx.mean()], axis=1)
ll["sum"] = ll.dn + ll.dx
ll.sort_values("sum")